In [44]:
import subprocess
import os
import numpy as np
import sys

sys.path.append(os.path.join(os.getcwd()))
sys.path.append(os.path.join(os.getcwd(), 'domain'))
from tests.domain.primary.domain.create_domain import DomainTables

mesh_list = [
  (2, 'rectangles.msh'),
  (2, 'triangles.msh'),
  (3, 'cube.msh'),
  (3, 'tetrahedron.msh'),
  (3, 'tetrahedron_big.msh'),
]

mpi_exec = "/usr/bin/mpirun"
python_exec = "/home/aben-ham/anaconda3/envs/work/bin/python3"
float_precision = 'float32'
dim, mesh_name = mesh_list[3]

def create_partitions(nb_partitions, mesh_name, float_precision, dim):
  root_file = os.getcwd()
  mesh_file_path = os.path.join(root_file, 'mesh', mesh_name)
  script_path = os.path.join(root_file, 'domain', 'create_partitions_mpi_worker.py')
  cmd = [mpi_exec, "-n", str(nb_partitions), "--oversubscribe", python_exec, script_path, mesh_file_path, float_precision, str(dim)]

  result = subprocess.run(cmd, env=os.environ.copy(), stderr=subprocess.PIPE)
  if result.returncode != 0:
    print(result.__str__(), os.getcwd())
    raise SystemExit(result.returncode)

domain_tables = DomainTables(nb_partitions=4, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)
unified_domain = DomainTables(nb_partitions=1, mesh_name=mesh_name, float_precision=float_precision, dim=dim, create_par_fun=create_partitions)

path /home/aben-ham/Desktop/work/manapy/tests/domain/primary/mesh/tetrahedron.msh precision float32 dim 3 rank 0
====> Start <=====
====> Creating Mesh <=====
gmsh:physical => triangle
gmsh:physical => tetra
node_cellid
Execution time: 0.005955 seconds
cell_cellnid
Execution time: 0.013250 seconds
cell_cellfid and boundary_faces
Execution time: 0.021076 seconds
node_bfid
Execution time: 0.021458 seconds
Start creating sub domains
Parse Input 
Start Creating SubDomains 
Make n part
Start dense to csr
Nb_vertices = 6000
size = 22800
End dense to csr
End METIS_PartGraphKway
End function
Create local cells
Create Physical faces And node old name
Create local domains struct
Create local domains struct p=0
Create map_bf_recv, shared_bf_recv
Create Tables
End Create Tables
Cells, CellsType, CellsLocToGlob
PhyFaces, PhyFacesName
Halo neighsub, Halos
End Halo neighsub, Halos
Create local domains struct p=1
Create map_bf_recv, shared_bf_recv
Create Tables
End Create Tables
Cells, CellsType, Cell

In [19]:
import os
import sys
sys.path.append(os.path.join(os.getcwd()))
sys.path.append(os.path.join(os.getcwd(), 'domain'))
from domain.create_domain import Domain, Mesh, GlobalDomain, LocalDomain, SingleCoreDomainTables
import numpy as np

mesh_list = [
  (2, 'rectangles.msh'),
  (2, 'triangles.msh'),
  (3, 'cube.msh'),
  (3, 'tetrahedron.msh'),
  (3, 'tetrahedron_big.msh'),
]
float_precision = 'float32' # the test does not support float64 or int64 yet
root_file = os.getcwd()
dim, mesh_path = mesh_list[0] # also modify dim variable accordingly
mesh_path = os.path.join(root_file, 'mesh', mesh_path) #tests/domain/primary/mesh

def create_domain(nb_parts):
  mesh = Mesh(mesh_path, dim)
  domain = GlobalDomain(mesh, float_precision)
  local_domain_data = domain.c_create_sub_domains(nb_parts)

  local_domains = LocalDomain.create_local_domains(local_domain_data)
  domains = [Domain(local_domains[i]) for i in range(len(local_domains))]

  return domains, SingleCoreDomainTables(domains, float_precision)

l_domains, domain_tables = create_domain(1)
g_domains, unified_domain = create_domain(1)

bounds
Execution time: 0.000121 seconds
node_cellid
Execution time: 0.000227 seconds
cell_cellnid
Execution time: 0.000270 seconds
_create_info
Execution time: 0.000533 seconds
Create cell volume and center
Execution time: 0.000620 seconds
Face measure face center face normal
Execution time: 0.000780 seconds
cell_halonid, face_haloid, node_haloid
Execution time: 0.000802 seconds
Create face and node names
Execution time: 0.001038 seconds
Create shared_ghost_info
Execution time: 0.001089 seconds
Create shared_ghost_info
Execution time: 0.001200 seconds
_create_halo_ghost_tables
Execution time: 0.001249 seconds
face_gradient_info
Execution time: 0.001378 seconds
_create_halo_ghost_tables
Execution time: 0.001417 seconds
_update_boundaries
Execution time: 0.001640 seconds
_define_BCs
Execution time: 0.001661 seconds
_create_normal_face_of_cell_2d
Execution time: 0.001726 seconds
_dist_ortho_function_2d
Execution time: 0.001816 seconds
bounds
Execution time: 0.000099 seconds
node_cellid
Ex

In [13]:
#Tetra
import importlib

module = importlib.import_module("helpers.TablesTestTetra3D")
importlib.reload(module)
TablesTestTetra3D = getattr(module, "TablesTestTetra3D")

module = importlib.import_module("helpers.TetraChecker3D")
importlib.reload(module)
TetraChecker3D = getattr(module, "TetraChecker3D")

d_cell_loctoglob = domain_tables.d_cell_loctoglob
g_cell_nodeid = unified_domain.d_cell_nodeid[0]
test_tables = TablesTestTetra3D(float_precision, d_cell_loctoglob, g_cell_nodeid)
test_tables.init()

checker = TetraChecker3D(decimal_precision=4, domain_tables=domain_tables, unified_domain=unified_domain, test_tables=test_tables)
checker.test_cell_info()
checker.test_node_info()
checker.test_face_info()
checker.test_halo_info()
checker.summary()



# ============================== #
# ========== Summary =========== #
# ============================== #
===> 21 [P] 0 [F] Total 21
---------------------------------------------------------
---------------------------------------------------------
### Entities Details
[P] 1/21: Cell Vertices
[P] 2/21: Cell Center
[P] 3/21: Cell Area
[P] 4/21: Cell Neighbors by face
[P] 5/21: Cell Neighbors by node
[P] 6/21: Cell Ghostnid *
[P] 7/21: Cell Number of cells
[P] 8/21: Node Cellid
[P] 9/21: Node Oldname
[P] 10/21: Node Name
[P] 11/21: Node ghostid, ghostcenter and ghostfaceinfo *
[P] 12/21: Node Number of nodes
[P] 13/21: Face Vertices
[P] 14/21: Face Measure
[P] 15/21: Face Center
[P] 16/21: Face Name
[P] 17/21: Face Oldname
[P] 18/21: Face Normal(Only abs)
[P] 19/21: Face CellId
[P] 20/21: Face Ghostcenter
[P] 21/21: Face Number of faces

---------------------------------------------------------
---------------------------------------------------------
===> 21 [P] 0 [F] Total 21


True

In [18]:
# Triangle
import importlib

module = importlib.import_module("helpers.TablesTestTriangles2D")
importlib.reload(module)
TestTablesTriangles2D = getattr(module, "TablesTestTriangles2D")

module = importlib.import_module("helpers.Checker2D")
importlib.reload(module)
Checker2D = getattr(module, "Checker2D")

d_cell_loctoglob = domain_tables.d_cell_loctoglob
g_cell_nodeid = unified_domain.d_cell_nodeid[0]
test_tables = TestTablesTriangles2D(float_precision, d_cell_loctoglob, g_cell_nodeid)
test_tables.init()

checker = Checker2D(decimal_precision=4, domain_tables=domain_tables, unified_domain=unified_domain, test_tables=test_tables)
checker.test_cell_info()
checker.test_face_info()
checker.test_node_info()
checker.test_halo_info()
checker.summary()


# ============================== #
# ========== Summary =========== #
# ============================== #
===> 22 [P] 0 [F] Total 22
---------------------------------------------------------
---------------------------------------------------------
### Entities Details
[P] 1/22: Cell Vertices
[P] 2/22: Cell Center
[P] 3/22: Cell Area
[P] 4/22: Cell Neighbors by face
[P] 5/22: Cell Neighbors by node
[P] 6/22: Cell Ghostnid *
[P] 7/22: Cell Number of cells
[P] 8/22: Face Vertices
[P] 9/22: Face Measure
[P] 10/22: Face Center
[P] 11/22: Face Name
[P] 12/22: Face Oldname
[P] 13/22: Face Normal(Only abs)
[P] 14/22: Face CellId
[P] 15/22: Face Ghostcenter
[P] 16/22: Face Cell face normal
[P] 17/22: Face Number of faces
[P] 18/22: Node Cellid
[P] 19/22: Node Oldname
[P] 20/22: Node Name
[P] 21/22: Node ghostid, ghostcenter and ghostfaceinfo *
[P] 22/22: Node Number of nodes

---------------------------------------------------------
---------------------------------------------------------
===

True

In [20]:
# Rectangle
import importlib

module = importlib.import_module("helpers.TablesTestRect2D")
importlib.reload(module)
TestTablesRect2D = getattr(module, "TablesTestRect2D")

module = importlib.import_module("helpers.Checker2D")
importlib.reload(module)
Checker2D = getattr(module, "Checker2D")

d_cell_loctoglob = domain_tables.d_cell_loctoglob
g_cell_nodeid = unified_domain.d_cell_nodeid[0]
test_tables = TestTablesRect2D(float_precision, d_cell_loctoglob, g_cell_nodeid)
test_tables.init()

checker = Checker2D(decimal_precision=4, domain_tables=domain_tables, unified_domain=unified_domain, test_tables=test_tables)
checker.test_cell_info()
checker.test_face_info()
checker.test_node_info()
checker.test_halo_info()
checker.summary()



# ============================== #
# ========== Summary =========== #
# ============================== #
===> 22 [P] 0 [F] Total 22
---------------------------------------------------------
---------------------------------------------------------
### Entities Details
[P] 1/22: Cell Vertices
[P] 2/22: Cell Center
[P] 3/22: Cell Area
[P] 4/22: Cell Neighbors by face
[P] 5/22: Cell Neighbors by node
[P] 6/22: Cell Ghostnid *
[P] 7/22: Cell Number of cells
[P] 8/22: Face Vertices
[P] 9/22: Face Measure
[P] 10/22: Face Center
[P] 11/22: Face Name
[P] 12/22: Face Oldname
[P] 13/22: Face Normal(Only abs)
[P] 14/22: Face CellId
[P] 15/22: Face Ghostcenter
[P] 16/22: Face Cell face normal
[P] 17/22: Face Number of faces
[P] 18/22: Node Cellid
[P] 19/22: Node Oldname
[P] 20/22: Node Name
[P] 21/22: Node ghostid, ghostcenter and ghostfaceinfo *
[P] 22/22: Node Number of nodes

---------------------------------------------------------
---------------------------------------------------------
===

True

In [16]:
# Cube
import importlib

module = importlib.import_module("helpers.TablesTestHexa3D")
importlib.reload(module)
TestTablesRect2D = getattr(module, "TablesTestHexa3D")

module = importlib.import_module("helpers.Checker3D")
importlib.reload(module)
Checker3D = getattr(module, "Checker3D")

d_cell_loctoglob = domain_tables.d_cell_loctoglob
g_cell_nodeid = unified_domain.d_cell_nodeid[0]
test_tables = TestTablesRect2D(float_precision, d_cell_loctoglob, g_cell_nodeid)
test_tables.init()

checker = Checker3D(decimal_precision=4, domain_tables=domain_tables, unified_domain=unified_domain, test_tables=test_tables)
checker.test_cell_info()
checker.test_face_info()
checker.test_node_info()
checker.test_halo_info()
checker.summary()



# ============================== #
# ========== Summary =========== #
# ============================== #
===> 21 [P] 0 [F] Total 21
---------------------------------------------------------
---------------------------------------------------------
### Entities Details
[P] 1/21: Cell Vertices
[P] 2/21: Cell Center
[P] 3/21: Cell Area
[P] 4/21: Cell Neighbors by face
[P] 5/21: Cell Neighbors by node
[P] 6/21: Cell Ghostnid *
[P] 7/21: Cell Number of cells
[P] 8/21: Face Vertices
[P] 9/21: Face Measure
[P] 10/21: Face Center
[P] 11/21: Face Name
[P] 12/21: Face Oldname
[P] 13/21: Face Normal(Only abs)
[P] 14/21: Face CellId
[P] 15/21: Face Ghostcenter
[P] 16/21: Face Number of faces
[P] 17/21: Node Cellid
[P] 18/21: Node Oldname
[P] 19/21: Node Name
[P] 20/21: Node ghostid, ghostcenter and ghostfaceinfo *
[P] 21/21: Node Number of nodes

---------------------------------------------------------
---------------------------------------------------------
===> 21 [P] 0 [F] Total 21


True